In [1]:
#Realizar upload dos arquivos e configuração da plataforma databricks antes de iniciar o processo

In [2]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
sqlContext

Out[ 3 ]: <pyspark.sql.context.HiveContext at 0x7f4c45910860>

In [4]:
#Importação pacotes necessários

from pyspark.context import SparkContext
from pyspark.sql.functions import regexp_extract
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum


In [5]:
#Adicionando os arquivos e carregando para o banco df_0

file1 = "/FileStore/tables/NASA_access_log_Aug95.gz"
file2 = "/FileStore/tables/NASA_access_log_Jul95.gz"
file =[file1,file2]
df_0 = spark.read.text(file)

In [6]:
# Olhando o banco de dados df_0

df_0.printSchema()

root
-- value: string (nullable = true)

In [7]:
#Observação: cada linha do arquivo foi carregada como uma coluna única de nome value
#Verificando o tipo de df_0

type(df_0)

Out[ 7 ]: pyspark.sql.dataframe.DataFrame

In [8]:
#Olhando as 20 primeiras linhas do dataframe

df_0.show(20, truncate=False)


+------------------------------------------------------------------------------------------------------------------------------+
value |
+------------------------------------------------------------------------------------------------------------------------------+
199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245 |
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985 |
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085 |
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0 |
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179 |
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0 |
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0 |
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985 |
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985 |
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074 |
unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310 |
unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786 |
unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /images/KSC-logosmall.gif HTTP/1.0" 200 1204 |
d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310 |
d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786 |
d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /images/KSC-logosmall.gif HTTP/1.0" 200 1204 |
129.94.144.152 - - [01/Jul/1995:00:00:17 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0 |
199.120.110.21 - - [01/Jul/1995:00:00:17 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713 |
ppptky391.asahi-net.or.jp - - [01/Jul/1995:00:00:18 -0400] "GET /facts/about_ksc.html HTTP/1.0" 200 3977 |
net-1-141.eden.com - - [01/Jul/1995:00:00:19 -0400] "GET /shuttle/missions/sts-71/images/KSC-95EC-0916.jpg HTTP/1.0" 200 34029|
+------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [9]:
# Verificando o tamanho de df_0
(df_0.count(), len(df_0.columns))

Out[ 9 ]: (3461613, 1)

In [10]:
#Configurando as expressões regulares para extração de dados:

re_host = r'(^\S+\.[\S+\.]+\S+)\s' #Host
re_timestamp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]' # Timestamp
re_url = r'\"(\S+)\s(\S+)\s*(\S*)\"' #url -Requisição 
re_codigo = r'\s(\d{3})\s' # codigo http
re_bytes = r'\s(\d+)$' # bytes

In [11]:
#Usando as expressões regulares para se obter um novo dataframe

df_1 = df_0.select(regexp_extract('value', re_host, 1).alias('host'),
                         regexp_extract('value', re_timestamp, 1).alias('timestamp'),
                         regexp_extract('value', re_url, 2).alias('url'),
                         regexp_extract('value', re_codigo, 1).cast('integer').alias('codigo'),
                         regexp_extract('value', re_bytes, 1).cast('integer').alias('bytes'))

#Visualizando o dataframe

df_1.show(20, truncate=True)
print((df_1.count(), len(df_1.columns)))

+--------------------+--------------------+--------------------+------+-----+
 host| timestamp| url|codigo|bytes|
+--------------------+--------------------+--------------------+------+-----+
 199.72.81.55|01/Jul/1995:00:00...| /history/apollo/| 200| 6245|
unicomp6.unicomp.net|01/Jul/1995:00:00...| /shuttle/countdown/| 200| 3985|
 199.120.110.21|01/Jul/1995:00:00...|/shuttle/missions...| 200| 4085|
 burger.letters.com|01/Jul/1995:00:00...|/shuttle/countdow...| 304| 0|
 199.120.110.21|01/Jul/1995:00:00...|/shuttle/missions...| 200| 4179|
 burger.letters.com|01/Jul/1995:00:00...|/images/NASA-logo...| 304| 0|
 burger.letters.com|01/Jul/1995:00:00...|/shuttle/countdow...| 200| 0|
 205.212.115.106|01/Jul/1995:00:00...|/shuttle/countdow...| 200| 3985|
 d104.aa.net|01/Jul/1995:00:00...| /shuttle/countdown/| 200| 3985|
 129.94.144.152|01/Jul/1995:00:00...| /| 200| 7074|
unicomp6.unicomp.net|01/Jul/1995:00:00...|/shuttle/countdow...| 200|40310|
unicomp6.unicomp.net|01/Jul/1995:00:00...|/images/NASA-logo...| 200| 786|
unicomp6.unicomp.net|01/Jul/1995:00:00...|/images/KSC-logos...| 200| 1204|
 d104.aa.net|01/Jul/1995:00:00...|/shuttle/countdow...| 200|40310|
 d104.aa.net|01/Jul/1995:00:00...|/images/NASA-logo...| 200| 786|
 d104.aa.net|01/Jul/1995:00:00...|/images/KSC-logos...| 200| 1204|
 129.94.144.152|01/Jul/1995:00:00...|/images/ksclogo-m...| 304| 0|
 199.120.110.21|01/Jul/1995:00:00...|/images/launch-lo...| 200| 1713|
ppptky391.asahi-n...|01/Jul/1995:00:00...|/facts/about_ksc....| 200| 3977|
 net-1-141.eden.com|01/Jul/1995:00:00...|/shuttle/missions...| 200|34029|
+--------------------+--------------------+--------------------+------+-----+
only showing top 20 rows

(3461613, 5)

In [12]:
#Verificando o número de linhas com null no dataframe original

df_0.filter(df_0['value'].isNull()).count()


Out[ 12 ]: 0

In [13]:
# Verificando o número de linhas com alguma coluna nula

df_1_null = df_1.filter(df_1['host'].isNull()| 
                             df_1['timestamp'].isNull()| 
                             df_1['url'].isNull()|
                             df_1['codigo'].isNull()|
                             df_1['bytes'].isNull())
                            
df_1_null.count()

Out[ 13 ]: 33905

In [14]:
#Verificando em quais colunas estão os nulos

def count_null(colunas):
    return spark_sum(col(colunas).isNull().cast('integer')).alias(colunas)

tabela = [count_null(colunas) for colunas in df_1.columns]

df_1.agg(*tabela).show()

+----+---------+---+------+-----+
host|timestamp|url|codigo|bytes|
+----+---------+---+------+-----+
 0| 0| 0| 1|33905|
+----+---------+---+------+-----+

In [15]:
# Encontrando a linha onde o codigo é nulo

df_0.filter(~df_0['value'].rlike(r'\s(\d{3})\s')).show()


+--------+
 value|
+--------+
alyssa.p|
+--------+

In [16]:
# Essa linha será excluída pois está incompleta, usando um editor de texto como o notepad++ é possível observar que esta é a última linha do arquivo de julho
# Como só existe uma única linha com código nulo vamos excluir essa linha

df_2 = df_1[df_1['codigo'].isNotNull()]

In [17]:
#Verificando o tamanho de df_2
df_2.count()

Out[ 17 ]: 3461612

In [18]:
#Os valores nulo se encontram agora na última coluna, como se trata de bytes vou substituir esses nulos por 0
df_3 = df_2.na.fill({'bytes': 0})

In [19]:
#Verificando que não existem mais colunas nulas

tabela = [count_null(colunas) for colunas in df_3.columns]
df_3.agg(*tabela).show()


+----+---------+---+------+-----+
host|timestamp|url|codigo|bytes|
+----+---------+---+------+-----+
 0| 0| 0| 0| 0|
+----+---------+---+------+-----+

In [20]:
df_3.cache()

Out[ 20 ]: DataFrame[host: string, timestamp: string, url: string, codigo: int, bytes: int]

In [21]:
# salvando o arquivo

df_4 = df_3.select("*").toPandas()
df_4.to_csv("/dbfs/FileStore/dados.csv", sep=',',header = True)